In [1]:
import pandas as pd
# anno_1 = '../Human_anno/humananno_res/lsy_scene.xlsx'
# anno_2 = '../Human_anno/humananno_res/lsy_scene.xlsx'

# anno_1oc =pd.read_excel(anno_1, usecols=['Model','scene'])
# anno_2oc =pd.read_excel(anno_2, usecols=['Model','scene'])
# anno_1oc = anno_1oc.astype(object).where(pd.notnull(anno_1oc), None)
# anno_2oc = anno_2oc.astype(object).where(pd.notnull(anno_2oc), None)

In [2]:
import numpy as np

def rank_numbers(numbers):
    sorted_indices = sorted(range(len(numbers)), key=lambda k: numbers[k], reverse=True)
    ranks = [0] * len(numbers)

    i = 0
    while i < len(sorted_indices):
        value_indices = [i]
        while i + 1 < len(sorted_indices) and numbers[sorted_indices[i]] == numbers[sorted_indices[i + 1]]:
            i += 1
            value_indices.append(i)
        average_rank = np.mean([index + 1 for index in value_indices])
        for index in value_indices:
            ranks[sorted_indices[index]] = average_rank
        i += 1

    return ranks

def calculate_spearman_manual(values1, values2):
    n = len(values1)
    m = len(values2)
    rank1 = rank_numbers(values1)
    rank2 = rank_numbers(values2)
    d = np.array(rank1) - np.array(rank2)
    d_squared = np.square(d)
    spearman_corr = 1 - (6 * np.sum(d_squared)) / (n * (n**2 - 1))
    return spearman_corr

def rank_basescore(base_score, draw_ratio):
    # 计算最大值和最小值的差值
    score_range = max(base_score) - min(base_score)
    # 计算平序的阈值
    draw_gap = draw_ratio * score_range

    # 对列表进行排序并保留原始索引
    indexed_scores = list(enumerate(base_score))
    indexed_scores.sort(key=lambda x: x[1])

    # 处理平序
    ranks = [0] * len(base_score)
    current_rank = 1
    for i in range(len(indexed_scores)):
        if i > 0 and abs(indexed_scores[i][1] - indexed_scores[i - 1][1]) < draw_gap:
            ranks[indexed_scores[i][0]] = current_rank
        else:
            current_rank = i + 1
            ranks[indexed_scores[i][0]] = current_rank

    return ranks

In [3]:
dimension = 'overall_consistency'

In [4]:
import json
jsonpath = '../Human_anno/{}.json'.format(dimension)
with open(jsonpath,'r') as f:
    oc = json.load(f)

# history ="../GPT4o_eval_results/{}_gpt4eval_results.json".format(dimension)
# with open(history,'r') as f:
#     gpt4o_eval_history = json.load(f)

In [5]:
# gptvsanno1_spearman = 0
# gptvsanno2_spearman = 0
# anno1vsanno2_spearman = 0
# gptvsannomean = 0
# badeval = []

# for i in range(length):
#     if i % 3 == 0:
#         gpt4o_eval_rs = np.array(list(oc[i]['gpt4o_eval'].values()))
#         human_anno = np.array(list(oc[i]['human_anno'].values()))
#         anno1 = human_anno[:,0]
#         anno2 = human_anno[:,1]
#         annomean = (anno1 + anno2)/2
#     else:
#         gpt4o_eval_rs+= np.array(list(oc[i]['gpt4o_eval'].values()))
#         human_anno = np.array(list(oc[i]['human_anno'].values()))
#         anno1 += human_anno[:,0]
#         anno2 += human_anno[:,1]
#         annomean += (anno1 + anno2)/2

#     if i % 3 == 2:
#         gptvsanno1_spearman += calculate_spearman_manual(gpt4o_eval_rs,anno1)
#         gptvsanno2_spearman += calculate_spearman_manual(gpt4o_eval_rs,anno2)
#         anno1vsanno2_spearman += calculate_spearman_manual(anno1,anno2)
#         gptvsannomean += calculate_spearman_manual(gpt4o_eval_rs,annomean)

# gptvsanno1_spearman = 3*gptvsanno1_spearman/length
# gptvsanno2_spearman = 3*gptvsanno2_spearman/length
# anno1vsanno2_spearman = 3*  anno1vsanno2_spearman/length
# gptvsannomean = 3*gptvsannomean/length
# print("GPT vs Anno1 Spearman: ",gptvsanno1_spearman)
# print("GPT vs Anno2 Spearman: ",gptvsanno2_spearman)
# print("Anno1 vs Anno2 Spearman: ",anno1vsanno2_spearman)
# print("GPT vs AnnoMean Spearman: ",gptvsannomean)

In [6]:
models = ['cogvideox5b','gen3', 'kling','videocrafter2', 'pika', 'show1', 'lavie']
# models = ['videocrafter2', 'pika', 'show1', 'lavie']
# models = ['cogvideox5b','gen3', 'kling']
idexls = []
# for i in range(0,len(oc),3):
#     idexls.append(i)
for i in range(1,len(oc),3):
    idexls.append(i)
length = len(idexls)

In [7]:
gptvsanno1_spearman = np.zeros(length)
gptvsanno2_spearman = np.zeros(length)
anno1vsanno2_spearman = np.zeros(length)
gptvsannomean_spearman = np.zeros(length)
baselinevsanno1_spearman = np.zeros(length)
baselinevsanno2_spearman = np.zeros(length)
baselinevsannomean_spearman = np.zeros(length)

gptscore = np.zeros([len(models)])
anno1score = np.zeros([len(models)])
anno2score = np.zeros([len(models)])
annomeanscore =np.zeros([len(models)])
baseline_rank = np.zeros([7])
badeval = []

for j in range(length):
    i = idexls[j]
    gpt4o_eval_rs = np.array(list(oc[i]['gpt4o_eval'].values()))
    human_anno = np.array(list(oc[i]['human_anno'].values()))
    # baseline_score = np.array(list(oc[i]['baseline_score'].values()))


    anno1 = human_anno[:,0]
    # anno2 = human_anno[:,1]
    anno2 = human_anno[:,3]

    # gpt4o_eval_rs[gpt4o_eval_rs == 4] = 5
    # anno1[anno1 == 4] = 5
    # anno2[anno2 == 4] = 5
    # gpt4o_eval_rs[gpt4o_eval_rs == 2] =1
    # anno1[anno1 == 2] = 1
    # anno2[anno2 == 2] = 1

    annomean = (anno1 + anno2)/2


    gptscore += gpt4o_eval_rs
    anno1score += anno1
    anno2score += anno2
    annomeanscore += annomean
    # baseline_rank += rank_basescore(baseline_score, 0.5)


    gptvsanno1 = calculate_spearman_manual(gpt4o_eval_rs,anno1)
    gptvsanno2 = calculate_spearman_manual(gpt4o_eval_rs,anno2)
    anno1vsanno2 = calculate_spearman_manual(anno1,anno2)
    gptvsannomean = calculate_spearman_manual(gpt4o_eval_rs,annomean)
    # baselinevsanno1 = calculate_spearman_manual(baseline_rank,anno1)
    # baselinevsanno2 = calculate_spearman_manual(baseline_rank,anno2)
    # baselinevsannomean = calculate_spearman_manual(baseline_rank,annomean)

    # j = i
    gptvsanno1_spearman[j] = gptvsanno1
    gptvsanno2_spearman[j] = gptvsanno2
    anno1vsanno2_spearman[j] = anno1vsanno2
    gptvsannomean_spearman[j] = gptvsannomean
    # baselinevsanno1_spearman[j] = baselinevsanno1
    # baselinevsanno2_spearman[j] = baselinevsanno2
    # baselinevsannomean_spearman[j] = baselinevsannomean

    if gptvsanno1 <0.2:
        badeval.append(i)

gptscore = gptscore/length
anno1score = anno1score/length
anno2score = anno2score/length
annomeanscore = annomeanscore/length

print("GPT vs Anno1 Spearman: ",gptvsanno1_spearman.mean())
print("GPT vs Anno2 Spearman: ",gptvsanno2_spearman.mean())
print("Anno1 vs Anno2 Spearman: ",anno1vsanno2_spearman.mean())
print("GPT vs AnnoMean Spearman: ",gptvsannomean_spearman.mean())
print("Baseline vs Anno1 Spearman: ",baselinevsanno1_spearman.mean())
print("Baseline vs Anno2 Spearman: ",baselinevsanno2_spearman.mean())
print("Baseline vs AnnoMean Spearman: ",baselinevsannomean_spearman.mean())


print("GPT average score: ",gptscore)
print("Anno1 average score: ",anno1score)
print("Anno2 average score: ",anno2score)
print("AnnoMean average score: ",annomeanscore)

GPT vs Anno1 Spearman:  0.7324862637362637
GPT vs Anno2 Spearman:  0.4252845368916798
Anno1 vs Anno2 Spearman:  0.4672782574568288
GPT vs AnnoMean Spearman:  0.6555631868131868
Baseline vs Anno1 Spearman:  0.0
Baseline vs Anno2 Spearman:  0.0
Baseline vs AnnoMean Spearman:  0.0
GPT average score:  [4.41758242 4.15934066 3.56043956 3.38461538 3.35164835 3.5
 2.65934066]
Anno1 average score:  [4.56043956 4.31868132 4.04395604 3.86813187 3.35164835 3.68681319
 3.07142857]
Anno2 average score:  [3.15934066 2.7967033  2.79120879 2.3956044  2.20879121 2.32967033
 1.77472527]
AnnoMean average score:  [3.85989011 3.55769231 3.41758242 3.13186813 2.78021978 3.00824176
 2.42307692]


In [8]:
from collections import Counter
badeval_prompt = []
for i in badeval:
    # print("spearman",gptvsannomean_spearman[i])
# print("videos",oc[i]['videos']['gen2'])
    # print("prompt",oc[i]['prompt_en'])
    badeval_prompt.append(oc[i]['prompt_en'])
    print(i%3,i,oc[i]['prompt_en'],gptvsanno2_spearman[i])
    # print("GPT score",oc[i]['gpt4o_eval'])
    # print("Anno score",oc[i]['human_anno'])
    # print("GPT reasons",gpt4o_eval_history[str(i)])

0 69 Time lapse of sunrise on mars. 0.9196428571428571


IndexError: index 231 is out of bounds for axis 0 with size 182

In [9]:
conuter = Counter(badeval_prompt)
repeated = {k:v for k,v in conuter.items() if v>1}

In [10]:
# print(sum(repeated.values()))
# print(repeated)
# with open('D:\Astudying\VideoEval\HighlyHumanLikeVEB\Human_anno\BadEval4OverallConsistency.json','w') as f:
#     json.dump(repeated,f,indent=4)

In [11]:
"""
1 7 A blue bicycle 0.0267857142857143
1 19 A pink bicycle 0.0803571428571429
1 22 A black bicycle 0.0982142857142857
2 44 A purple car 0.1964285714285714
1 49 A black car 0.1696428571428571
1 55 A red bird -0.0535714285714286
0 63 A yellow bird -0.0625
2 65 A yellow bird -0.1160714285714286
1 73 A pink bird 0.044642857142857095
2 92 A yellow cat -0.2857142857142858
1 94 A red umbrella 0.0357142857142857
2 107 An orange umbrella -0.03571428571428581
0 114 A black umbrella 0.0803571428571429
1 133 An orange suitcase 0.0803571428571429
2 134 An orange suitcase 0.1517857142857143
0 135 A purple suitcase 0.1785714285714286
0 147 A red bowl 0.1339285714285714
2 149 A red bowl 0.0535714285714286
2 179 A green chair -0.0625
2 191 A purple chair 0.0982142857142857
0 192 A pink chair 0.008928571428571397
1 199 A white chair 0.1964285714285714
0 204 A green clock 0.1785714285714286
1 205 A green clock 0.1071428571428571
0 210 A yellow clock 0.0892857142857143
2 218 A purple clock -0.0535714285714286
2 233 A green vase 0.044642857142857095
1 247 A pink vase -0.09821428571428581
1 250 A black vase 0.0535714285714286"""

'\n1 7 A blue bicycle 0.0267857142857143\n1 19 A pink bicycle 0.0803571428571429\n1 22 A black bicycle 0.0982142857142857\n2 44 A purple car 0.1964285714285714\n1 49 A black car 0.1696428571428571\n1 55 A red bird -0.0535714285714286\n0 63 A yellow bird -0.0625\n2 65 A yellow bird -0.1160714285714286\n1 73 A pink bird 0.044642857142857095\n2 92 A yellow cat -0.2857142857142858\n1 94 A red umbrella 0.0357142857142857\n2 107 An orange umbrella -0.03571428571428581\n0 114 A black umbrella 0.0803571428571429\n1 133 An orange suitcase 0.0803571428571429\n2 134 An orange suitcase 0.1517857142857143\n0 135 A purple suitcase 0.1785714285714286\n0 147 A red bowl 0.1339285714285714\n2 149 A red bowl 0.0535714285714286\n2 179 A green chair -0.0625\n2 191 A purple chair 0.0982142857142857\n0 192 A pink chair 0.008928571428571397\n1 199 A white chair 0.1964285714285714\n0 204 A green clock 0.1785714285714286\n1 205 A green clock 0.1071428571428571\n0 210 A yellow clock 0.0892857142857143\n2 218 A p

In [12]:
"""0 18 A pink bicycle 0.4732142857142857
1 22 A black bicycle 0.0982142857142857
2 29 A red car 0.2232142857142857
0 30 A green car 0.7142857142857143
0 39 An orange car 0.7946428571428572
2 44 A purple car 0.1964285714285714
2 56 A red bird 0.375
0 57 A green bird 0.2857142857142857
0 63 A yellow bird -0.0625
1 73 A pink bird 0.044642857142857095
0 78 A white bird 0.5982142857142857
0 81 A black cat 0.4107142857142857
1 88 An orange cat 0.2410714285714286
2 89 An orange cat 0.4107142857142857
2 92 A yellow cat -0.2857142857142858
0 96 A green umbrella 0.3303571428571429
1 121 A red suitcase 0.7857142857142857
0 126 A blue suitcase 0.2232142857142857
1 151 A green bowl 0.6517857142857143
2 179 A green chair -0.0625
0 180 A blue chair 0.5535714285714286
1 181 A blue chair 0.3571428571428571
0 198 A white chair 0.5446428571428572
2 212 A yellow clock 0.2767857142857143
1 220 A pink clock 0.6785714285714286
0 225 A white clock 0.2589285714285714
1 226 A white clock 0.375
2 236 A blue vase 0.4285714285714286
1 250 A black vase 0.0535714285714286"""

'0 18 A pink bicycle 0.4732142857142857\n1 22 A black bicycle 0.0982142857142857\n2 29 A red car 0.2232142857142857\n0 30 A green car 0.7142857142857143\n0 39 An orange car 0.7946428571428572\n2 44 A purple car 0.1964285714285714\n2 56 A red bird 0.375\n0 57 A green bird 0.2857142857142857\n0 63 A yellow bird -0.0625\n1 73 A pink bird 0.044642857142857095\n0 78 A white bird 0.5982142857142857\n0 81 A black cat 0.4107142857142857\n1 88 An orange cat 0.2410714285714286\n2 89 An orange cat 0.4107142857142857\n2 92 A yellow cat -0.2857142857142858\n0 96 A green umbrella 0.3303571428571429\n1 121 A red suitcase 0.7857142857142857\n0 126 A blue suitcase 0.2232142857142857\n1 151 A green bowl 0.6517857142857143\n2 179 A green chair -0.0625\n0 180 A blue chair 0.5535714285714286\n1 181 A blue chair 0.3571428571428571\n0 198 A white chair 0.5446428571428572\n2 212 A yellow clock 0.2767857142857143\n1 220 A pink clock 0.6785714285714286\n0 225 A white clock 0.2589285714285714\n1 226 A white cloc